In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from gmplot import GoogleMapPlotter
from random import random
import csv

In [2]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open('apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])

    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))

        f.write('\t\t};\n')
        f.write(
            '\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')

    def color_scatter(self, lats, lngs, values=None, colormap='coolwarm',
                      size=None, marker=False, s=None, **kwargs):
        def rgb2hex(rgb):
            """ Convert RGBA or RGB to #RRGGBB """
            rgb = list(rgb[0:3]) # remove alpha if present
            rgb = [int(c * 255) for c in rgb]
            hexcolor = '#%02x%02x%02x' % tuple(rgb)
            return hexcolor

        if values is None:
            colors = [None for _ in lats]
        else:
            cmap = plt.get_cmap(colormap)
            norm = Normalize(vmin=min(values), vmax=max(values))
            scalar_map = ScalarMappable(norm=norm, cmap=cmap)
            colors = [rgb2hex(scalar_map.to_rgba(value)) for value in values]
        for lat, lon, c in zip(lats, lngs, colors):
            self.scatter(lats=[lat], lngs=[lon], c=c, size=size, marker=marker,
                         s=s, **kwargs)

    def cam_tower_scatter(self, lats, lngs, color=None, size=None, marker=True, c=None, s=None, symbol='o',
                          tower_color=None, **kwargs):
        color = color or c
        size = size or s or 40
        kwargs["color"] = color
        kwargs["size"] = size
        settings = self._process_kwargs(kwargs)
        for lat, lng in zip(lats[:-1], lngs[:-1]):
            if marker:
                self.marker(lat, lng, settings['color'])
            else:
                self._add_symbol(Symbol(symbol, lat, lng, size), **settings)
        if marker:
            self.marker(lats[-1], lngs[-1], tower_color)
            tower_col = self.color_dict.get(color)
            tower_col = self.html_color_codes.get(color,color)
            self.points.append((lats[-1], lngs[-1], tower_col[1:], "no implementation"))

In [19]:
norm_lats = []
norm_lons = []
with open('./object_positions.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        norm_lats.append(float(row['latp1']))
        norm_lons.append(float(row['lonp1'])*(-1.))
print('norm_lats:',norm_lats,'\nnorm_lons:',norm_lons)

norm_lats: [38.59739685404832, 38.598482609532915, 38.598816539317795, 38.599090849028975, 38.59914417185073, 38.59914417185073, 38.59914417185073, 38.59933219523949, 38.599689857757895] 
norm_lons: [-90.64442142469932, -90.64455151789481, -90.64470975068606, -90.64462242125359, -90.64458381928326, -90.64458381928326, -90.64458381928326, -90.64461507283222, -90.64487795294762]


In [20]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(norm_lats[0], norm_lons[0], initial_zoom,
                              map_type='satellite')
gmap.scatter(norm_lats,norm_lons,alpha=0.5,c='red', marker=True)
gmap.draw("norm_map5.html")

In [11]:
depth_lats = []
depth_lons = []
with open('./object_positions.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        depth_lats.append(float(row['latp']))
        depth_lons.append(float(row['lonp'])*(-1.))
print('depth_lats:',depth_lats,'\ndepth_lons:',depth_lons)

depth_lats: [38.596677793292635, 38.597130939824154, 38.59735337371244, 38.59741223707159, 38.59741296436111, 38.59741295019673, 38.597412940560716, 38.59749726874073, 38.59780360158391] 
depth_lons: [-90.64009297090905, -90.64047710514045, -90.64069490378199, -90.64074763809417, -90.64074710927805, -90.64074707788718, -90.6407470565321, -90.64085774147402, -90.64116144240052]


In [14]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(depth_lats[0], depth_lons[0], initial_zoom,
                              map_type='satellite')
gmap.scatter(depth_lats,depth_lons,alpha=0.5,c='red', marker=True)
gmap.draw("depth_map2.html")